In [5]:
days=365
filename =f"stock_nifty_next50_{days}.csv"
#filename ="stock_nifty_next50.csv"
#filename ="stock.csv"
import pandas as pd

# Load the Excel file
df = pd.read_csv(f"../../inputdata/trading/{filename}")


In [4]:



#stragety 1




# Prepare date and sort
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(['SYMBOL', 'TIMESTAMP']).reset_index(drop=True)

results = []

for symbol in df['SYMBOL'].unique():
    sdf = df[df['SYMBOL'] == symbol].copy()
    sdf = sdf.sort_values('TIMESTAMP').reset_index(drop=True)
    sdf['Prev_Close'] = sdf['CLOSING_PRICE'].shift(1)
    sdf['Up'] = sdf['CLOSING_PRICE'] > sdf['Prev_Close']
    sdf['Up_1'] = sdf['Up'].shift(1)
    sdf['Buy_Signal'] = sdf['Up'] & sdf['Up_1']
    profit = 0
    trades = 0
    for i, row in sdf[sdf['Buy_Signal']].iterrows():
        buy_idx = i
        sell_idx = i + 2
        if sell_idx < len(sdf):
            buy_price = sdf.loc[buy_idx, 'OPENING_PRICE']
            sell_price = sdf.loc[sell_idx, 'CLOSING_PRICE']
            trade_profit = (sell_price - buy_price) / buy_price * 100000  # ₹1 lakh per trade
            profit += trade_profit
            trades += 1
    results.append({'SYMBOL': symbol, 'Total Profit (₹)': profit, 'Trades': trades})

results_df = pd.DataFrame(results)

results_df.to_csv(f"../../output/trading/processed_{filename}", index=False)
results_df = results_df.sort_values('Total Profit (₹)', ascending=False)
print(results_df.head(10))  # Show top 10
print("\nMost profitable symbol:", results_df.iloc[0]['SYMBOL'], "with profit ₹", round(results_df.iloc[0]['Total Profit (₹)'],2))


        SYMBOL  Total Profit (₹)  Trades
29        LTIM      97142.932166      63
4   BAJAJ-AUTO      91347.507729      66
22   ICICIBANK      79433.482002      77
5   BAJAJFINSV      79285.174675      63
23  INDUSINDBK      77229.426015      60
16     HCLTECH      76603.221840      55
18    HDFCLIFE      74399.814031      65
15      GRASIM      74285.240344      54
8         BPCL      74134.630531      70
42         TCS      71705.330778      54

Most profitable symbol: LTIM with profit ₹ 97142.93


In [11]:

#starategy 2


df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(['SYMBOL', 'TIMESTAMP']).reset_index(drop=True)

results = []

# Process each symbol separately
for symbol in df['SYMBOL'].unique():
    sdf = df[df['SYMBOL'] == symbol].copy()
    sdf = sdf.sort_values('TIMESTAMP').reset_index(drop=True)
    
    # Calculate previous close and up days
    sdf['Prev_Close'] = sdf['CLOSING_PRICE'].shift(1)
    sdf['Up'] = sdf['CLOSING_PRICE'] > sdf['Prev_Close'] # Up by at least 1% 
    sdf['Up_1'] = sdf['Up'].shift(1)
    
    # 10-day average volume
    sdf['Avg_Vol_10'] = sdf['TOT_TRADED_QTY'].rolling(window=10).mean()
    
    # Buy signal: 2 consecutive up closes + volume surge (>1.2x avg volume)
    sdf['Buy_Signal'] = (sdf['Up'] & sdf['Up_1']) & (sdf['TOT_TRADED_QTY'] > 1.2 * sdf['Avg_Vol_10'])

    profit = 0
    trades = 0
    for i, row in sdf[sdf['Buy_Signal']].iterrows():
        # Buy on the next day and hold for 3 days
        print(f"Processing symbol: {symbol}, Buy signal at date {sdf.loc[i, 'TIMESTAMP']}")
        buy_idx = i
        sell_idx = i + 3  # Hold 3 days
        if sell_idx < len(sdf):
            buy_price = sdf.loc[buy_idx, 'OPENING_PRICE']
            sell_price = sdf.loc[sell_idx, 'CLOSING_PRICE']
            trade_profit = (sell_price - buy_price) / buy_price * 100000  # ₹1 lakh invested
            profit += trade_profit
            trades += 1
    
    if trades > 0:
        results.append({'SYMBOL': symbol, 'Total Profit (₹)': profit, 'Trades': trades})

# Sort by total profit descending
results_df = pd.DataFrame(results).sort_values('Total Profit (₹)', ascending=False)
results_df.to_csv(f'../../output/trading/processed_{filename}', index=False)
print(results_df.head(10))  # Top 10 symbols by profit

best = results_df.iloc[0]

print(f"\nMost profitable symbol: {best['SYMBOL']} with total profit ₹{round(best['Total Profit (₹)'],2)} from {best['Trades']} trades")


#sum_profit = results_df['Total Profit (₹)'].sum()
print("\nTotal profit across all symbols: ₹", round(results_df['Total Profit (₹)'].sum(), 2))


Processing symbol: ADANIGREEN, Buy signal at date 2024-07-16 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-08-01 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-08-19 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-09-17 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-09-20 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-09-23 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-11-06 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-11-29 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2024-12-02 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2025-01-15 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2025-01-16 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2025-03-21 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2025-04-24 00:00:00
Processing symbol: ADANIGREEN, Buy signal at date 2025-05-05 00:00:00
Processing symbol: A